In [29]:
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, random_split, DataLoader
import numpy as np
import pandas as pd
import copy
import math
import torch.nn.functional as F
import sys
import csv
from torch.nn.functional import normalize

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Read Dataset
read pain prediction dataset into classification problem.

In [40]:
data_folder = r"C:\Users\ronny\Documents\GitHub\MetaPain\Data\biosignals_filtered_Processed\eda"

def get_pain_prediction(data_folder):

    """
    Returns the dataset loaders for all tasks of pain prediction dataset
    """

    files = os.listdir(data_folder)
    num_tasks = len(files)
    with open(data_folder+'/'+ files[0],'r') as f:
        pickle_in = list(csv.reader(f, delimiter=','))
        feature,target = np.array(pickle_in)[:,:-1].astype(float),np.array(pickle_in)[:,-1]
        ind = ~np.isnan(feature).any(axis=1)
        feature = feature[ind, :]
        target = target[ind]
        meta_ind = (target == "PA1")|(target == "PA2")|(target == "PA3")
        local_ind = (target == "BL1")|(target == "PA4")
        feature_local = feature[local_ind,:]
        target_local = target[local_ind]
        feature_meta = feature[meta_ind,:]
        target_meta = target[meta_ind]
        
    for task_id in range(1,num_tasks):

#         with open(os.path.join(data_folder, files[task_id-1]),'r') as f:
        with open(data_folder+'/'+ files[task_id],'r') as f:

            feature,target = np.array(pickle_in)[:,:-1].astype(float),np.array(pickle_in)[:,-1]
            ind = ~np.isnan(feature).any(axis=1)
            feature = feature[ind, :]
            target = target[ind]
            meta_ind = (target == "PA1")|(target == "PA2")|(target == "PA3")
            local_ind = (target == "BL1")|(target == "PA4")
            feature_local = np.concatenate((feature_local,feature[local_ind,:]))
            target_local = np.concatenate((target_local,target[local_ind]))
            feature_meta = np.concatenate((feature_meta,feature[meta_ind,:]))
            target_meta = np.concatenate((target_meta,target[meta_ind]))
            
    feature_meta = normalize(torch.Tensor(feature_meta),dim=0)
    feature_local = normalize(torch.Tensor(feature_local),dim=0)
    target_meta = np.unique(target_meta, return_inverse=True)[1].astype(float)
    target_local = np.unique(target_local, return_inverse=True)[1].astype(float)
    dataset_local = TensorDataset(feature_local,torch.Tensor(target_local))
    train_dataset_local, test_dataset_local = random_split(dataset_local, 
                                                           [int(len(target_local)*0.80), (len(target_local)-int(len(target_local)*0.80))],
                                                           generator=torch.Generator().manual_seed(42))
    dataset_meta = TensorDataset(feature_meta,torch.Tensor(target_meta))
    train_dataset_meta, test_dataset_meta = random_split(dataset_meta, 
                                                           [int(len(target_meta)*0.80), (len(target_meta)-int(len(target_meta)*0.80))],
                                                           generator=torch.Generator().manual_seed(42)) 
    datasets_local = {'train': train_dataset_local, 'test': test_dataset_local}
    datasets_meta = {'train': train_dataset_meta, 'test': test_dataset_meta}

    return datasets_local,datasets_meta

datasets_local,datasets_meta = get_pain_prediction(data_folder)

# Network Construction

Construct multilayer neural network for the pain prediction dataset with input dimension of 22.

In [46]:
class MLP(nn.Module):
    """
    Two layer MLP for pain prediction dataset benchmarks.
    """
    def __init__(self, hiddens,dropout):
        super().__init__()
        self.W1 = nn.Linear(22, hiddens)
        self.W2 = nn.Linear(hiddens, hiddens)
        self.W3 = nn.Linear(hiddens, hiddens)
        self.W4 = nn.Linear(hiddens, 2)
        self.norm1 = nn.BatchNorm1d(hiddens)
        self.norm2 = nn.BatchNorm1d(hiddens)
        self.norm3 = nn.BatchNorm1d(hiddens)
        self.relu = nn.ReLU(inplace=True)
        self.dropout_1 = nn.Dropout(p=dropout)
        self.dropout_2 = nn.Dropout(p=dropout)
        self.dropout_3 = nn.Dropout(p=dropout)
        
    def forward(self,x):
        
        out = self.W1(x)
        out = self.relu(out)
        out = self.norm1(out)
        out = self.dropout_1(out)
        out = self.W2(out)
        out = self.relu(out)
        out = self.norm2(out)
        out = self.dropout_2(out)
        out = self.W3(out)
        out = self.relu(out)
        out = self.norm3(out)
        out = self.dropout_3(out)
        out = self.W4(out)
        
        return out   
        

# Initiate meta training using PA1-PA3

In [47]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [48]:
def train_epoch(model, optimizer, loader, criterion):
    
    train_acc = AverageMeter()
    train_loss = AverageMeter()
    
    model = model.to(DEVICE)
    model.train()
    for _,(data, target) in enumerate(loader):
        data = data.to(DEVICE)
        target = target.type(torch.LongTensor).to(DEVICE)
        optimizer.zero_grad()
        
        pred = model(data)
        loss = criterion(pred.view(-1,2), target)
        train_loss.update(loss.item())
        
        acc = (pred.argmax(dim=-1) == target).float().mean()
        train_acc.update(acc.item())
        
        loss.backward()
        optimizer.step()
        
    return {'loss': train_loss.avg, 'Accuracy': train_acc.avg}

        
def eval_epoch(model, loader, criterion):
    
    test_acc = AverageMeter()
    test_loss = AverageMeter()
    
    model = model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        for data, target in loader:
            data = data.to(DEVICE)
            target = target.type(torch.LongTensor).to(DEVICE)
            
            pred = model(data)
            loss = criterion(pred, target)
            test_loss.update(loss.item())
            
            acc = (pred.argmax(dim=-1) == target).float().mean()
            test_acc.update(acc.item())
            
    return {'loss': test_loss.avg, 'Accuracy': test_acc.avg}
            
        
        

In [49]:
train_loader = DataLoader(datasets_local['train'], batch_size=64, shuffle=True)
test_loader = DataLoader(datasets_local['test'], batch_size=64, shuffle=True)

criterion = nn.CrossEntropyLoss().to(DEVICE)
model = MLP(64, 0.5)

lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_model = model.to(DEVICE)
best_acc = 0
epoch = 50

for _ in np.arange(epoch):
    
    train_epoch(model, optimizer, train_loader, criterion)
    metric = eval_epoch(model, test_loader, criterion)
    
    print(f"validation loss is {metric['loss']}, validation accuracy is {metric['Accuracy']}")
    
    if metric['Accuracy'] > best_acc:
        best_model = copy.deepcopy(model)
        best_acc = metric['Accuracy']
    

validation loss is 0.6110656261444092, validation accuracy is 0.9474431818181818
validation loss is 0.14755834571339868, validation accuracy is 0.9744318181818182
validation loss is 0.04383430748500607, validation accuracy is 1.0
validation loss is 0.019353784874758938, validation accuracy is 1.0
validation loss is 0.009158547036349773, validation accuracy is 1.0
validation loss is 0.004473472268066623, validation accuracy is 1.0
validation loss is 0.002520426536317576, validation accuracy is 1.0
validation loss is 0.001775235530327667, validation accuracy is 1.0
validation loss is 0.005226738930849189, validation accuracy is 1.0
validation loss is 0.0006065659584816207, validation accuracy is 1.0
validation loss is 0.0007644741298546168, validation accuracy is 1.0
validation loss is 0.00034577116523657673, validation accuracy is 1.0
validation loss is 0.00019004443575712767, validation accuracy is 1.0
validation loss is 0.00016187796295112506, validation accuracy is 1.0
validation los